# Multichannel operation with EIS

This notebook is an example in which [external potentiostats](https://zahner.de/products#external-potentiostats) such as PP2x2, XPOT2 or EL1002 are controlled both standalone (SCPI) and as an EPC device on a [Zennium series instrument](https://zahner.de/products#potentiostats). Up to 16 external potentiostats can share one Zennium to use it for impedance measurements for example.

**This notebook cannot be executed and has been created only for documentation and explanation of the source code, because Jupyter does not support loops over multiple cells.**

Knowledge of all other notebooks of this repository, the [Remote2 manual](https://doc.zahner.de/manuals/remote2.pdf) and the [zahner_potentiostat package](https://github.com/Zahner-elektrik/Zahner-Remote-Python) is assumed as known.

For this example a Zennium with [EPC card](https://zahner.de/products-details/addon-cards/epc42) is necessary, to this EPC card the external potentiostats must be connected with the appropriate cable. A maximum of 4 potentiostats per card and a maximum of 4 cards are possible.  
The external potentiostats and the Zennium must also be connected to the computer via USB cable separately.

In this example, a PP242 and an XPOT2 are used to cyclically charge and discharge capacitors, and after a defined number of cycles, impedance is measured with the Zennium via the EPC interface.

The [ImpedanceRampHotSwap.ipynb](https://github.com/Zahner-elektrik/Thales-Remote-Python/tree/main/Examples/ImpedanceRampHotSwap/ImpedanceRampHotSwap.ipynb) example is a supplement to this example, there the potentiostat is not switched off when switching between EPC and SCPI.

**Important Notes:**

Each external potentiostat needs its own thread or process for the measurement. This requires a basic understanding of multithreading and thread synchronization. This example is solved with threads. Because of the [threading](https://docs.python.org/3/library/threading.html#module-threading) concept with the [Python Global Interpreter Lock (GIL)](https://docs.python.org/3/glossary.html#term-global-interpreter-lock) it can make sense or be necessary to use [multiprocessing](https://docs.python.org/3/library/multiprocessing.html) instead of [threading](https://docs.python.org/3/library/threading.html#module-threading). If multiprocessing is used, then the used lock with which the shared Zennium is synchronized has to be changed to the [multiprocessing lock](https://docs.python.org/3/library/multiprocessing.html#multiprocessing.Lock) instead of the [threading lock](https://docs.python.org/3/library/threading.html#lock-objects).

**Only one potentiostat, i.e. thread/process, can access the Zennium at a time.**

In [ ]:
from thales_remote.epc_scpi_handler import EpcScpiHandlerFactory,EpcScpiHandler
from thales_remote.script_wrapper import PotentiostatMode
from zahner_potentiostat.scpi_control.datahandler import DataManager
from zahner_potentiostat.display.onlinedisplay import OnlineDisplay
from zahner_potentiostat.scpi_control.datareceiver import TrackTypes

import threading
from datetime import datetime

# Name creation function

This function simplifies the naming of the cells/channels.

In [ ]:
def getFileName(channel,cycle):
    time = str(datetime.now().time())
    time = time.replace(":","")
    time = time.replace(",","")
    time = time.replace(".","")
    return f"channel{channel}_cycle{cycle}" # + time

# Thread for channel 1

The following function is executed as a thread for channel one.
The same function could also be executed as a thread for each channel, but here different sequences are to be measured with each external potentiostat, therefore two different functions are used.

Only the first thread is explained in more detail, the second thread is only slightly different from the first thread.

The threads unfortunately have to be defined before the main function, where the initialization of the different devices takes place and [EpcScpiHandler](https://doc.zahner.de/thales_remote/epc_scpi_handler.html#thales_remote.epc_scpi_handler.EpcScpiHandler) objects are created for each external potentiostat.

An [EpcScpiHandler](https://doc.zahner.de/thales_remote/epc_scpi_handler.html#thales_remote.epc_scpi_handler.EpcScpiHandler) object is then passed to the respective thread as a parameter, so that the thread can work with this object and the measurement process can be programmed.

In [ ]:
def channel1Thread(deviceHandler, channel = 0):
    deviceHandler.scpiInterface.setMaximumTimeParameter(15)
    deviceHandler.scpiInterface.setParameterLimitCheckToleranceTime(0.1)
    
    configuration = {
        "figureTitle":"Online Display Channel 1",
        "xAxisLabel":"Time",
        "xAxisUnit":"s",
        "xTrackName":TrackTypes.TIME.toString(),
        "yAxis":
            [{"label": "Voltage", "unit": "V", "trackName":TrackTypes.VOLTAGE.toString()},
             {"label": "Current", "unit": "A", "trackName":TrackTypes.CURRENT.toString()}]
    }
    
    for i in range(3):
        filename = getFileName(channel = channel, cycle = i)   

First the device is in SCPI standalone mode and a capacitor is charged and discharged 2 times.

During the measurement, the online display is also started for visualization of voltage and current.

In [ ]:
        onlineDisplay = OnlineDisplay(deviceHandler.scpiInterface.getDataReceiver(), displayConfiguration=configuration)
        
        deviceHandler.scpiInterface.measureOCVScan()
        
        for n in range(2):
            deviceHandler.scpiInterface.measureCharge(current = 1e-3,
                                                      stopVoltage = 2,
                                                      maximumTime = "5 min")
            
            deviceHandler.scpiInterface.measureDischarge(current = -1e-3,
                                                         stopVoltage = 0.5,
                                                         maximumTime = "5 min")

After cycling, the EPC mode must be activated in order to control the potentiostat as an EPC device with the Zennium. For this purpose the lock must be aquired.  
In order to be able to continue measuring the OCP, the lock is acquired with *blocking = False*, so if the Zennium is occupied and [acquireSharedZennium()](https://doc.zahner.de/thales_remote/epc_scpi_handler.html#thales_remote.epc_scpi_handler.EpcScpiHandler.acquireSharedZennium) returns *False*, the OCP is measured again for 15 seconds.  
This is repeated until the zennium is no longer used by another channel and is available.

In [ ]:
        deviceHandler.scpiInterface.setMaximumTimeParameter(15)    
        while deviceHandler.acquireSharedZennium(blocking = False) == False:
            deviceHandler.scpiInterface.measureOCVScan()

Now that the Zennium is reserved for this measurement, the measurement data is saved as text and the online display is closed.

In [ ]:
        dataManager = DataManager(deviceHandler.scpiInterface.getDataReceiver())
        dataManager.saveDataAsText(filename + ".txt")
        
        onlineDisplay.close()
        del onlineDisplay

To use the external potentiostat as an EPC device, the device must be switched to EPC mode as shown below.

The change between SCPI and EPC interface must be initiated by the currently active controller, it is not possible to "get back" the control via SCPI in EPC mode.

In [ ]:
        deviceHandler.switchToEPC()

From now on the Zennium is controlled by Remote2 and the [Thales-Remote-Python library](https://github.com/Zahner-elektrik/Thales-Remote-Python), the [switchToEPC()](https://doc.zahner.de/thales_remote/epc_scpi_handler.html#thales_remote.epc_scpi_handler.EpcScpiHandler.switchToEPC) function automatically selects the EPC channel to which the device is connected.

The object *deviceHandler.scpiInterface* loses its validity after calling the function [switchToEPC()](https://doc.zahner.de/thales_remote/epc_scpi_handler.html#thales_remote.epc_scpi_handler.EpcScpiHandler.switchToEPC) and **can not be used anymore**.

In the [other example](https://github.com/Zahner-elektrik/Thales-Remote-Python/tree/main/Examples/ImpedanceRampHotSwap/ImpedanceRampHotSwap.ipynb) the parameter *keepPotentiostatState = True* is used to keep the potentiostat switched on when switching the operation mode.

In [ ]:
        deviceHandler.sharedZenniumInterface.setEISNaming("individual")
        deviceHandler.sharedZenniumInterface.setEISOutputPath(r"C:\THALES\temp\multichannel")
        deviceHandler.sharedZenniumInterface.setEISOutputFileName(filename)
        
        deviceHandler.sharedZenniumInterface.setPotentiostatMode(PotentiostatMode.POTMODE_POTENTIOSTATIC)
        deviceHandler.sharedZenniumInterface.setAmplitude(10e-3)
        deviceHandler.sharedZenniumInterface.setPotential(0)
        deviceHandler.sharedZenniumInterface.setLowerFrequencyLimit(100)
        deviceHandler.sharedZenniumInterface.setStartFrequency(500)
        deviceHandler.sharedZenniumInterface.setUpperFrequencyLimit(1000)
        deviceHandler.sharedZenniumInterface.setLowerNumberOfPeriods(5)
        deviceHandler.sharedZenniumInterface.setLowerStepsPerDecade(2)
        deviceHandler.sharedZenniumInterface.setUpperNumberOfPeriods(20)
        deviceHandler.sharedZenniumInterface.setUpperStepsPerDecade(5)
        deviceHandler.sharedZenniumInterface.setScanDirection("startToMax")
        deviceHandler.sharedZenniumInterface.setScanStrategy("single")
        
        deviceHandler.sharedZenniumInterface.measureEIS()

After the measurement, the Zennium must be released that it can be used by other channels. This is realized with the method [switchToSCPIAndReleaseSharedZennium()](https://doc.zahner.de/thales_remote/epc_scpi_handler.html#thales_remote.epc_scpi_handler.EpcScpiHandler.switchToSCPIAndReleaseSharedZennium). Afterwards the device is available for standalone SCPI measurements without Thales.

The object *deviceHandler.sharedZenniumInterface* loses its validity after calling the function [switchToSCPIAndReleaseSharedZennium()](https://doc.zahner.de/thales_remote/epc_scpi_handler.html#thales_remote.epc_scpi_handler.EpcScpiHandler.switchToSCPIAndReleaseSharedZennium) and **can not be used anymore**.

In the [other example]((https://github.com/Zahner-elektrik/Thales-Remote-Python/tree/main/Examples/ImpedanceRampHotSwap/ImpedanceRampHotSwap.ipynb)) the parameter *keepPotentiostatState = True* is used to keep the potentiostat switched on when switching the operation mode.

In [ ]:
        deviceHandler.switchToSCPIAndReleaseSharedZennium()
    
    return

# Thread for channel 2

The following function is executed as a thread for channel 2.

In [ ]:
def channel2Thread(deviceHandler):
    deviceHandler.scpiInterface.setMaximumTimeParameter(15)
    deviceHandler.scpiInterface.setParameterLimitCheckToleranceTime(0.1)
    
    configuration = {
        "figureTitle":"Online Display Channel 2",
        "xAxisLabel":"Time",
        "xAxisUnit":"s",
        "xTrackName":TrackTypes.TIME.toString(),
        "yAxis":
            [{"label": "Voltage", "unit": "V", "trackName":TrackTypes.VOLTAGE.toString()},
             {"label": "Current", "unit": "A", "trackName":TrackTypes.CURRENT.toString()}]
    }
    
    for i in range(2):
        filename = getFileName(channel = 2, cycle = i)
               
        onlineDisplay = OnlineDisplay(deviceHandler.scpiInterface.getDataReceiver(), displayConfiguration=configuration)
        
        deviceHandler.scpiInterface.measureOCVScan()
        
        for n in range(2):
            deviceHandler.scpiInterface.measureCharge(current = 4,
                                                      stopVoltage = 1,
                                                      maximumTime = "5 min")
            
            deviceHandler.scpiInterface.measureDischarge(current = -4,
                                                         stopVoltage = 0.6,
                                                         maximumTime = "5 min")
        
        dataManager = DataManager(deviceHandler.scpiInterface.getDataReceiver())
        dataManager.saveDataAsText(filename + ".txt")
        
        onlineDisplay.close()
        del onlineDisplay

In this example we simply wait without measurements until the Zennium is available, therefore no parameters are necessary for [acquireSharedZennium()](https://doc.zahner.de/thales_remote/epc_scpi_handler.html?highlight=acquiresharedzennium#thales_remote.epc_scpi_handler.EpcScpiHandler.acquireSharedZennium).  
The potentiostat should have been switched off before.

In [ ]:
        deviceHandler.acquireSharedZennium()              
        deviceHandler.switchToEPC()

        deviceHandler.sharedZenniumInterface.setEISNaming("individual")
        deviceHandler.sharedZenniumInterface.setEISOutputPath(r"C:\THALES\temp\multichannel")
        deviceHandler.sharedZenniumInterface.setEISOutputFileName(filename)
        
        deviceHandler.sharedZenniumInterface.setPotentiostatMode(PotentiostatMode.POTMODE_POTENTIOSTATIC)
        deviceHandler.sharedZenniumInterface.setAmplitude(10e-3)
        deviceHandler.sharedZenniumInterface.setPotential(0)
        deviceHandler.sharedZenniumInterface.setLowerFrequencyLimit(100)
        deviceHandler.sharedZenniumInterface.setStartFrequency(500)
        deviceHandler.sharedZenniumInterface.setUpperFrequencyLimit(1000)
        deviceHandler.sharedZenniumInterface.setLowerNumberOfPeriods(5)
        deviceHandler.sharedZenniumInterface.setLowerStepsPerDecade(2)
        deviceHandler.sharedZenniumInterface.setUpperNumberOfPeriods(20)
        deviceHandler.sharedZenniumInterface.setUpperStepsPerDecade(5)
        deviceHandler.sharedZenniumInterface.setScanDirection("startToMax")
        deviceHandler.sharedZenniumInterface.setScanStrategy("single")
        
        deviceHandler.sharedZenniumInterface.measureEIS()
        
        deviceHandler.switchToSCPIAndReleaseSharedZennium()
    
    return

# Initialization of the devices

Before the two threads with the measurement tasks are executed, the device management objects of type [EpcScpiHandler](https://doc.zahner.de/thales_remote/epc_scpi_handler.html#thales_remote.epc_scpi_handler.EpcScpiHandler) are created with the [EpcScpiHandlerFactory](https://doc.zahner.de/thales_remote/epc_scpi_handler.html#thales_remote.epc_scpi_handler.EpcScpiHandlerFactory) class.

The Zennium could also be connected to another computer via USB and you can control the Zennium over network, for this the default parameter *shared_zennium_target = "localhost"* of the constructor of [EpcScpiHandlerFactory](https://doc.zahner.de/thales_remote/epc_scpi_handler.html#thales_remote.epc_scpi_handler.EpcScpiHandlerFactory) must be overwritten with the corresponding IP address.
The external potentiostats can currently only be controlled via USB.

In [ ]:
if __name__ == "__main__":    
    handlerFactory = EpcScpiHandlerFactory()

With the method [createEpcScpiHandler()](https://doc.zahner.de/thales_remote/epc_scpi_handler.html#thales_remote.epc_scpi_handler.EpcScpiHandlerFactory.createEpcScpiHandler) of the [EpcScpiHandlerFactory](https://doc.zahner.de/thales_remote/epc_scpi_handler.html#thales_remote.epc_scpi_handler.EpcScpiHandlerFactory) object handlerFactory, a new [EpcScpiHandler](https://doc.zahner.de/thales_remote/epc_scpi_handler.html#thales_remote.epc_scpi_handler.EpcScpiHandler) object can be created, which is used to control the devices as explained above.

The method [createEpcScpiHandler()](https://doc.zahner.de/thales_remote/epc_scpi_handler.html#thales_remote.epc_scpi_handler.EpcScpiHandlerFactory.createEpcScpiHandler) has two parameters:  
* **epcChannel:** The number of the EPC channel to which the external potentiostat is connected.
* **serialNumber:** The serial number of the device to uniquely identify it.

In [ ]:
    XPOT2 = handlerFactory.createEpcScpiHandler(epcChannel=1, serialNumber=27000)
    PP242 = handlerFactory.createEpcScpiHandler(epcChannel=4, serialNumber=35000)

After the two devices are initialized, they are passed to the respective [threads](https://docs.python.org/3/library/threading.html#module-threading) and the threads are started. Then the main only waits until the two measurement threads are finished.

For the first thread a channel number is passed, this would be necessary for the naming of the files, if the same function is executed as different threads.

In [ ]:
    channel1ThreadHandler = threading.Thread(target=channel1Thread, args=(XPOT2,1))
    channel2ThreadHandler = threading.Thread(target=channel2Thread, args=(PP242,))
    
    channel1ThreadHandler.start()
    channel2ThreadHandler.start()
    
    channel1ThreadHandler.join()
    channel2ThreadHandler.join()
    
    handlerFactory.closeAll()
    print("finish")